In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

In [3]:
# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

In [4]:
war_df = mss_df.loc[mss_df['UniqueID'].str.contains("WAR")].copy()

In [5]:
client_df = war_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

client_df['Reporting Period'] = client_df['Reporting Period'].fillna(0)
client_df.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
client_df['trial'] = client_df.index.map(s)
client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'trial']
client_df['Reporting Period'] = pd.to_numeric(client_df['Reporting Period'], downcast='integer')
client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)

client_df.drop(columns=['trial'],inplace=True)


I am thinking of creating more tables : county, state, origin, species. That way if we have to update the way somethign is spelled we just do it in one place.

Maybe even tract as well?

In [6]:
species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))


In [7]:
origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))


In [8]:
ga_intake_df = war_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # Does this need included? At the moment we do funny stuff with total gt and subtracting future gt to get GA current rp (above)
                       'Notes'
                       ]].copy()

# Do I want to include a Ticketid column here or use a crossreference table?
# Should reporting period be in here?

In [9]:
status_df = war_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

In [10]:
def intake_breaker(lst):
    intake_lst = []
    activity_lst = []
    for ele in lst:
        intake_lst.append(ele[:9])
        activity_lst.append(ele[9:11])
        intake_st = set(intake_lst)
        activity_st = set(activity_lst)
        
    return intake_lst, activity_lst, intake_st, activity_st


def ticketid(lst):
    """This function creates a ticket id based off a list of unique ids

    Args:
        lst (list): a list of unique ids that have the same Tract_Cmp_Stand from MSS
    """
    if len(lst) == 0:
        return "ERROR, Where is your uniqueid?"
    elif len(lst) < 2:
        return lst[0]
    elif len(lst) > 1:
        # Check to see if there are multiple intakes
        temp_lst, activity_lst, temp_st, activity_st = intake_breaker(lst)
        temp_lst.sort()
        activity_lst.sort()
        if len(temp_st) > 1:
            return f"{temp_lst[0]}XXXX"
        # If there are not multiple intakes. Check to see if there are multiple activities in the lst (Intake portion should be same for all)
        elif len(activity_st) > 1:
           return f"{temp_lst[0]}XXXX"
       # If there is only a single activity type
        elif len(activity_st) == 1:
            return f"{temp_lst[0]}{activity_lst[0]}XX"
        else:
            return "Failed within len(lst) > 1"
        
    else:
        return "Failed at length of lst" 

In [11]:
war_inprog = war_df[war_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST', 'PENDING'])].copy()
war_inprog['Tract'] = war_inprog['Tract'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['CMP'] = war_inprog['CMP'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['Stand'] = war_inprog['Stand'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['TCS'] = war_inprog['Tract'] + "_" + war_inprog['CMP'] + "_" + war_inprog['Stand']

# After talking with Rob, we need to make sure that we separate out the GA and the ARB Guidance
grouped_activities = war_inprog.groupby(['TCS', 'GuidanceProvider'])['UniqueID'].apply(list).reset_index()

trial_df = grouped_activities.copy()
trial_df['BBID'] = trial_df['UniqueID'].apply(lambda x: ticketid(x))

prev_combined = war_inprog.groupby('TicketID')['TCS'].apply(set).reset_index()

tcs = []
for val in prev_combined['TCS'].loc[prev_combined['TCS'].str.len()> 1].values:
    for item in val:
        tcs.append(item)
        
ticket_id_dict = {}
for i, row in prev_combined.loc[prev_combined['TCS'].str.len()> 1].iterrows():
    # print(row['TicketID']) # ticket id
    value_list = []
    for value in row['TCS']:
        value_list.append(value)
    ticket_id_dict[row['TicketID']] = value_list
    
for k, v in ticket_id_dict.items():
    trial_df.loc[trial_df['TCS'].isin(v), 'otherid'] = k

def final_id(row):
    if row['otherid'] != "nan":
        return row['otherid']
    else:
        return row['BBID']
    
trial_df['finalid'] = trial_df[['BBID', 'otherid']].apply(final_id, axis=1)

def break_list(x):
    trial_list = []
    for item in x:
        for iter in item:
            trial_list.append(iter)
            
    return trial_list

new_ticket_df = trial_df.groupby('finalid').agg({'TCS': lambda x: list(x),
                                 'UniqueID': lambda x: break_list(x)}).reset_index()

client_for_ticket = client_df.copy()
for id in new_ticket_df['finalid'].unique().tolist():
    client_for_ticket.loc[client_for_ticket['UniqueID'].isin(new_ticket_df.loc[new_ticket_df['finalid'] == id, 'UniqueID'].values[0]), 'TicketID'] = id

new_tcs_intake = client_for_ticket.groupby(['TicketID',
                                            'UniqueID'])['Client GT'].sum().reset_index()

old_ticket_ids = dict(zip(war_df['UniqueID'], war_df['TicketID']))
new_tcs_intake['old_ticketids'] = new_tcs_intake['UniqueID'].map(old_ticket_ids)
new_tcs_intake.loc[~new_tcs_intake['TicketID'].str.contains("WAR"), 'TicketID'] = new_tcs_intake['old_ticketids']

# Manually fix this issue where we had to deal with overlaps and SPB management
new_tcs_intake.loc[new_tcs_intake['UniqueID'] == 'WAR231006BI02', 'TicketID'] = "WAR230817CCXX"


ticketid_table = new_tcs_intake[['TicketID', 'UniqueID']]

## Getting the ticket table

In [ ]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "WAR_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "WAR_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'])

big_df['Date'] = pd.to_datetime(big_df['Date'])

mill_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

mill_df = mill_df.drop_duplicates()

P:\Mill Tickets\ARCHIVE\WAR_RP6
WAR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP7
WAR_RP7_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP8
WAR_RP8_Compiled.xlsm
P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP2
WAR_RP2_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP3
WAR_RP3_Compiled.xlsm


# Getting the expected harvest

Find all activities that are Open, Pending, In Process, Post

Get their ticket id

Get the sum of the expected rp8 for that ticketid
get the sum of the expected rp9 for that ticketid

get the harvested volumes for those ticketids (already did this by getting the sums and translating from old ticketid to new ticketid)

### First Step

Make the base expected table

I think we need to start this from the MSS_df

To get us values as stated on MSS:
- Get the Expected GT for the RP requested
- Get the Future GT
  - Add 1 to the RP requested
- Groupby the new Ticket ID

To Updated for RP9:
- get all ticket ids that are open in process, pending, post
- Get the harvested volume for those ticket ids (might have to walk from old ticketid to uniqueid to new ticketid)
- find any remaining expected for those ticket ids
  - Add remaining to the RP9 volume




Get harvested tons by old tid

In [13]:
harvested_tons_oldTID = mill_df[mill_df['Reporting Period'] == 8].groupby("TicketID")['Tons (U.S)'].sum().reset_index()

In [ ]:
harvested_tons_oldTID['Tons (U.S)'].sum()

Let's get new and old TIDs that are currently in process

In [15]:
# Getting harvest ids that were in RP 8 and are still "Active" (Potentially need updating due to harvest)
harvest_activities = war_df[(war_df['Reporting Period'] == 8) & (war_df['Status'].isin(['IN PROCESS', "POST"]))]['UniqueID'].unique().tolist()
harvest_old_ticketid = war_df[(war_df['Reporting Period'] == 8) & (war_df['Status'].isin(['IN PROCESS', "POST"]))]['TicketID'].unique().tolist()

In [ ]:
# Let's see how many tons are under old TIDs of activities in question
harvested_tons_oldTID[harvested_tons_oldTID['TicketID'].isin(harvest_old_ticketid)]['Tons (U.S)'].sum()

In [17]:
# Let's get the new TIDs based on these harvest activities
new_ticketIDs = ticketid_table[ticketid_table['UniqueID'].isin(harvest_activities)]['TicketID'].unique().tolist()


In [ ]:
# Let's see what is in old_ticketid that is not in new_ticketIDs
[item for item in harvest_old_ticketid if item not in new_ticketIDs]

Ok, so it looks like we don't have any old ticketids in the harvested file that are not in the new ticketids

That doesn't mean there aren't changes of uniqueids between the two sets of ticketids

In [ ]:
war_df[war_df['TicketID'] == "WAR220728XXXX"]['UniqueID'].unique().tolist()

In [ ]:
ticketid_table[ticketid_table['TicketID'] == "WAR220728XXXX"]

In [29]:
for id in harvest_old_ticketid:
    old_unique = war_df[war_df['TicketID'] == id]['UniqueID'].unique().tolist()
    new_unique = ticketid_table[ticketid_table['TicketID'] == id]['UniqueID'].unique().tolist()
    inold_notinew = [item for item in old_unique if item not in new_unique]
    if len(inold_notinew) > 0:
        print(id, old_unique)

Ok, so it looks like we are lucky and the new ticketids aren't messing up the accounting

## Next steps

- Get the expected GT for RP8 by new ticketid
- Get the expected GT for RP9 by new ticketid
- Update the expected RP9 GT with Rob's harvesting ruleset

RP8 Expected - will be the conservative between Client and GT (in war_df it will be the "Current RP Expected GT")


In [54]:
trial_expected = pd.merge(ticketid_table, war_df[['UniqueID', 'Current RP Expected GT', 'Status']], on='UniqueID', how='left')
trial_expected = pd.merge(trial_expected, client_df[['UniqueID', 'Reporting Period']], on='UniqueID', how='left')
war_rp8_expected = trial_expected[(trial_expected['Reporting Period'] == 8) & trial_expected['Status'].isin(['IN PROCESS', 'POST'])].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
war_rp8_expected = pd.merge(war_rp8_expected, harvested_tons_oldTID, on='TicketID', how='left')
war_rp8_expected['Remaining'] = war_rp8_expected['Current RP Expected GT'] - war_rp8_expected['Tons (U.S)']

In [ ]:
war_rp8_expected[war_rp8_expected['Remaining'] > 0]